In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import numpy as np

# = = = = = = = = = = = = = = =

import json

In [2]:
directory_in = './file/input'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：731



,No,Group,Type,Manufacturer,Part Number,Vehicle,OEM,Picture,Url,Note 1,...,Width (MM),Json_Src,Info,Json_Note 1,Json_Note 2,Json_Price,Json_Specification,Type Code,Part Code,JOIN_MPNTCPC
0,1,Heat & Air Conditioning,A/C Receiver Drier / Accumulator,UAC,HA10018C,FORD MUSTANG 1989-1993,F3ZZ19D734BA,"=DISPIMG(""ID_A99DF0B0109D403D9D750011C2721E81"",1)","https://www.rockauto.com/en/parts/uac,HA10018C...",with Hose Assembly,...,,"{""0"": ""https://www.rockauto.com/info/1063/HA 1...",https://www.rockauto.com/en/moreinfo.php?pk=98...,"{""0"": ""with Hose Assembly""}","{""0"": ""Category: A/C Receiver Drier / Accumula...","{""0"": ""$41.79""}",{},6972,9803464,UAC;HA10018C;6972;9803464
1,2,Heat & Air Conditioning,A/C Receiver Drier / Accumulator,UAC,HA10020C,FORD SABLE 1996-2001\nFORD TAURUS 1996-1999\nM...,F7DZ17C836EA;F7DZ19C836EA,"=DISPIMG(""ID_3BEBE05003AC42148C3CB584EC83B137"",1)","https://www.rockauto.com/en/parts/uac,HA10020C...",with Hose Assembly,...,,"{""0"": ""https://www.rockauto.com/info/1063/HA 1...",https://www.rockauto.com/en/moreinfo.php?pk=98...,"{""0"": ""with Hose Assembly""}","{""0"": ""Category: A/C Receiver Drier / Accumula...","{""0"": ""$27.79""}",{},6972,9803472,UAC;HA10020C;6972;9803472
2,3,Heat & Air Conditioning,A/C Receiver Drier / Accumulator,UAC,HA10021C,FORD E-150 1997-2006\nFORD E-250 1997-2006\nFO...,2C2419C913BB;2C2Z19C8368BB;2C2Z19C836AB;2C2Z19...,"=DISPIMG(""ID_79723599476F4BABB7625AF16008BA1D"",1)","https://www.rockauto.com/en/parts/uac,HA10021C...",with Hose Assembly,...,,"{""0"": ""https://www.rockauto.com/info/1063/HA 1...",https://www.rockauto.com/en/moreinfo.php?pk=98...,"{""0"": ""with Hose Assembly""}","{""0"": ""Category: A/C Receiver Drier / Accumula...","{""0"": ""$28.79""}",{},6972,9803476,UAC;HA10021C;6972;9803476
3,4,Heat & Air Conditioning,A/C Receiver Drier / Accumulator,UAC,HA10022C,FORD E-150 1997-2006\nFORD E-250 1997-2006\nFO...,2C2Z19C836AA;5C2Z19C386AB;5C2Z19C8368AB;5C2Z19...,"=DISPIMG(""ID_9E5D3F6440CB45D39898ABEA14528D31"",1)","https://www.rockauto.com/en/parts/uac,HA10022C...",with Hose Assembly,...,,"{""0"": ""https://www.rockauto.com/info/1063/HA 1...",https://www.rockauto.com/en/moreinfo.php?pk=98...,"{""0"": ""with Hose Assembly""}","{""0"": ""Category: A/C Receiver Drier / Accumula...","{""0"": ""$26.79""}","{""0"": {""0"": ""Inlet (MM)"", ""1"": ""22""}, ""1"": {""0...",6972,9803480,UAC;HA10022C;6972;9803480
4,5,Heat & Air Conditioning,A/C Receiver Drier / Accumulator,UAC,HA10029C,CHEVROLET CAMARO 1997\nFORD LTD II 1979\nPONTI...,02724937;2724937;88968524,"=DISPIMG(""ID_E41A5DE9C6C04C1C8341F2B25122922A"",1)","https://www.rockauto.com/en/parts/uac,HA10029C...",Hose Assembly and Accumulator Included,...,,"{""0"": ""https://www.rockauto.com/info/1063/HA 1...",https://www.rockauto.com/en/moreinfo.php?pk=98...,"{""0"": ""Hose Assembly and Accumulator Included""}","{""0"": ""Category: A/C Receiver Drier / Accumula...","{""0"": ""$24.79""}",{},6972,9803500,UAC;HA10029C;6972;9803500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,Heat & Air Conditioning,A/C Receiver Drier / Accumulator,UAC,RD9993C,CHEVROLET METRO 1998-2001\nGEO METRO 1995-1997...,5862040960;80351SX0961;JRJ100550,"=DISPIMG(""ID_557EA4FD3DFE4B08A3BB93DDBBF73473"",1)","https://www.rockauto.com/en/parts/uac,RD9993C,...",Drier Pad Mount,...,,"{""0"": ""https://www.rockauto.com/info/1063/RD 9...",https://www.rockauto.com/en/moreinfo.php?pk=98...,"{""0"": ""Drier Pad Mount""}","{""0"": ""Category: A/C Receiver Drier / Accumula...","{""0"": ""$7.29""}","{""0"": {""0"": ""Diameter (MM)"", ""1"": ""60""}, ""1"": ...",6972,9835536,UAC;RD9993C;6972;9835536
727,728,Heat & Air Conditioning,A/C Receiver Drier / Accumulator,UAC,RD9996C,ACURA RL 1996-2004\nDODGE VIPER 1992-2002\nLEX...,4708201;5264990AB;80351SN7003;80351SP0013;8847...,"=DISPIMG(""ID_736DD6E8B4EF4E5DA8DB106F6301A408"",1)","https://www.rockauto.com/en/parts/uac,RD9996C,...",Drier Pad Mount,...,,"{""0"": ""https://w

In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                   1
 Group                          Heat & Air Conditioning
 Type                  A/C Receiver Drier / Accumulator
 Manufacturer                                       UAC
 Part Number                                   HA10018C
                                     ...               
 Json_Price                             {"0": "$41.79"}
 Json_Specification                                  {}
 Type Code                                         6972
 Part Code                                      9803464
 JOIN_MPNTCPC                 UAC;HA10018C;6972;9803464
 Name: 0, Length: 94, dtype: object)

In [5]:
import time

data = {'search': crawler_series['Part Number'],
        'page': '1'}

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_check = f'''https://uacparts.com/Catalog/SearchIndex?search={crawler_series['Part Number']}'''

        resp = requests.post('https://uacparts.com/SearchAjaxList/Search',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(5, 15))

        if resp.status_code == 200:
            dict_ = resp.json()

            if dict_['d']['it'] == None and dict_['d']['m'] == False and dict_['d']['th'] == None and dict_['d']['p'] == None:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1


{'d': {'it': None,
  'm': False,
  'c': '\r<li id=\'part42125\' class="awe-li part awe-il" data-val="42125"><div class="label-bar">\r<h2 class="part-label">HA 10018C</h2></div><div>\r<div class="part-image">\r<button class="btnleft" style="padding: 2px 40px;position:relative;top:50%;left:0%;color:white;background:#003366;" onclick="slide(-1,42125)">&#10094;</button>\r<button class="btnright" style="padding: 2px 40px;position:relative;top:50%;right:0%;color:white;background:#003366;" onclick="slide(+1,42125)">&#10095;</button>\r<img src=\'https://www.uacparts.com/photo/WEBP/HA/HA 10018C_1_562.webp\' style="display:block;" onclick="LoadImage(42125)" class="thumb thumb-part mySlides_42125" onerror="this.onerror=null;this.src=\'/images/nophoto.jpg\'" alt=\'picture\' />\r<img src=\'https://www.uacparts.com/photo/WEBP/HA/HA 10018C_4_562.webp\' style="display:none;" onclick="LoadImage(42125)" class="thumb thumb-part mySlides_42125" onerror="this.onerror=null;this.src=\'/images/nophoto.jpg\'" 

In [6]:
soup = BeautifulSoup(dict_['d']['c'], 'lxml')
html = etree.HTML(str(soup))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <li class="awe-li part awe-il" data-val="42125" id="part42125">
   <div class="label-bar">
    <h2 class="part-label">
     HA 10018C
    </h2>
   </div>
   <div>
    <div class="part-image">
     <button class="btnleft" onclick="slide(-1,42125)" style="padding: 2px 40px;position:relative;top:50%;left:0%;color:white;background:#003366;">
      ❮
     </button>
     <button class="btnright" onclick="slide(+1,42125)" style="padding: 2px 40px;position:relative;top:50%;right:0%;color:white;background:#003366;">
      ❯
     </button>
     <img alt="picture" class="thumb thumb-part mySlides_42125" onclick="LoadImage(42125)" onerror="this.onerror=null;this.src='/images/nophoto.jpg'" src="https://www.uacparts.com/photo/WEBP/HA/HA 10018C_1_562.webp" style="display:block;"/>
     <img alt="picture" class="thumb thumb-part mySlides_42125" onclick="LoadImage(42125)" onerror="this.onerror=null;this.src='/images/nophoto.jpg'" src="https://www.uacparts.com/photo/WEBP/HA/HA 10018C_4_

In [7]:
list_li = html.xpath('//li[@class="awe-li part awe-il"]')

list_li

[<Element li at 0x125ae61c0>]

In [8]:
list_part_number = [li.xpath('./div[@class="label-bar"]/h2[@class="part-label"]/text()')[0] for li in list_li]

list_part_number

['HA 10018C']

In [9]:
array_index_part = np.where(np.array([part_number.replace(' ', '') for part_number in list_part_number]) == crawler_series['Part Number'])[0]

array_index_part

array([0])

In [10]:
if array_index_part.size == 1:
    index_part = array_index_part[0]

index_part

np.int64(0)

In [11]:
dict_src, list_src = {}, [href.strip() for href in list_li[index_part].xpath('./div/div[@class="part-image"]/img[@alt="picture"]/@src')]
for i, src in enumerate(list_src):
    dict_src[str(i)] = src

dict_src

{'0': 'https://www.uacparts.com/photo/WEBP/HA/HA 10018C_1_562.webp',
 '1': 'https://www.uacparts.com/photo/WEBP/HA/HA 10018C_4_562.webp'}

In [12]:
dict_note, list_text = {}, list_li[index_part].xpath('./div/div[@class="part-info"]/div[@class="comments"]/text()')
for i, text in enumerate(list_text):
    dict_note[str(i)] = text

dict_note

{'0': 'Accumulator with Hose Assembly'}

In [13]:
list_dict_correct.append({'Part Number': crawler_series['Part Number'],
                          '[UAC] Part Number': list_part_number[index_part],
                          '[UAC] Picture': '',
                          '[UAC] Url': url_check,
                          '[UAC] Json_Src': json.dumps(dict_src),
                          '[UAC] Json_Note': json.dumps(dict_note)})

list_dict_correct

[{'Part Number': 'HA10018C',
  '[UAC] Part Number': 'HA 10018C',
  '[UAC] Picture': '',
  '[UAC] Json_Src': '{"0": "https://www.uacparts.com/photo/WEBP/HA/HA 10018C_1_562.webp", "1": "https://www.uacparts.com/photo/WEBP/HA/HA 10018C_4_562.webp"}',
  '[UAC] Json_Note': '{"0": "Accumulator with Hose Assembly"}'}]

In [14]:
crawler_status = True

crawler_status

True

In [15]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                                    1
 Group                                           Heat & Air Conditioning
 Type                                   A/C Receiver Drier / Accumulator
 Manufacturer                                                        UAC
 Part Number                                                    HA10018C
                                             ...                        
 Json_Specification                                                   {}
 Type Code                                                          6972
 Part Code                                                       9803464
 JOIN_MPNTCPC                                  UAC;HA10018C;6972;9803464
 Check_Url             https://uacparts.com/Catalog/SearchIndex?searc...
 Name: 0, Length: 95, dtype: object]

In [16]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Part Number'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct

,Part Number,[UAC] Part Number,[UAC] Picture,[UAC] Json_Src,[UAC] Json_Note
0,HA10018C,HA 10018C,,"{""0"": ""https://www.uacparts.com/photo/WEBP/HA/...","{""0"": ""Accumulator with Hose Assembly""}"


In [17]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Part Number'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)

df_error

,No,Group,Type,Manufacturer,Part Number,Vehicle,OEM,Picture,Url,Note 1,...,Json_Src,Info,Json_Note 1,Json_Note 2,Json_Price,Json_Specification,Type Code,Part Code,JOIN_MPNTCPC,Check_Url
0,1,Heat & Air Conditioning,A/C Receiver Drier / Accumulator,UAC,HA10018C,FORD MUSTANG 1989-1993,F3ZZ19D734BA,"=DISPIMG(""ID_A99DF0B0109D403D9D750011C2721E81"",1)","https://www.rockauto.com/en/parts/uac,HA10018C...",with Hose Assembly,...,"{""0"": ""https://www.rockauto.com/info/1063/HA 1...",https://www.rockauto.com/en/moreinfo.php?pk=98...,"{""0"": ""with Hose Assembly""}","{""0"": ""Category: A/C Receiver Drier / Accumula...","{""0"": ""$41.79""}",{},6972,9803464,UAC;HA10018C;6972;9803464,https://uacparts.com/Catalog/SearchIndex?searc...
